# Linear Advection Equation

\begin{equation}
\begin{aligned}
    \partial_t u(t,x) + \partial_x (a(x) u(t,x)) &= 0, && t \in (0,T), x \in (x_{min}, x_{max}), \\
    u(0,x) &= u_0(x), && x \in (x_{min}, x_{max}), \\
    \text{boundary conditions}, &&& x \in \partial (x_{min}, x_{max}).
\end{aligned}
\end{equation}

The boundary conditions depend on the sign of the transport velocity $a$ at the boundary. In particular, specifying a Dirichlet type boundary condition is only allowed for inflow boundaries, e.g. $a(x_{min}) > 0$ at $x = x_{min}$.

In [ ]:
using Revise
using SummationByPartsOperators, OrdinaryDiffEq
using Plots, LaTeXStrings, Printf

# general parameters
xmin = -1.
xmax = +1.
tspan = (0., 8.0)
afunc(x) = one(x)
u0func(x) = sinpi(x)
# Dirichlet type boundary conditions; they are used only at inflow boundaries
left_bc(t) = t >= 3 ? sinpi(t) : zero(t)
right_bc(t) = zero(t)

# discretisation parameters
interior_order = 4
N = 101
# whether a split form should be applied or not
split_form = Val(false)

# setup spatial semidiscretisation
D = derivative_operator(MattssonSvärdShoeybi2008(), 1, interior_order, xmin, xmax, N)
# whether or not artificial dissipation should be applied: nothing, dissipation_operator(D)
Di = nothing
semidisc = VariableLinearAdvectionNonperiodicSemidiscretisation(D, Di, afunc, split_form, left_bc, right_bc)
ode = semidiscretise(u0func, semidisc, tspan)

# solve ode
sol = solve(ode, SSPRK104(), dt=D.Δx, adaptive=false, 
            saveat=range(first(tspan), stop=last(tspan), length=200))

# visualise the result
plot(xguide=L"x", yguide=L"u")
plot!(evaluate_coefficients(sol[end], semidisc), label="")

In [ ]:
# make a movie
anim = Animation()
idx = 1
x, u = evaluate_coefficients(sol[idx], semidisc)

fig = plot(x, u, xguide=L"x", yguide=L"u", xlim=extrema(x), ylim=(-1.05, 1.05),
           #size=(1024,768), dpi=250,
           label="", title=@sprintf("\$t = %6.2f \$", sol.t[idx]))
for idx in 1:length(sol.t)
    fig[1] = x, sol.u[idx]
    plot!(title=@sprintf("\$t = %6.2f \$", sol.t[idx]))
    frame(anim)
end
gif(anim)